# 1. Disciplina de Introdução a Banco de Dados - 2025/02
### Trabalho Prático 2: Análise de Dados de um Dataset de Alunos de Iniciação Científica

# 2. Membros (nome e número de matrícula)

- Arthur Augusto Paiva Lemos - 2023427414
- Filipe Henrique Nunes -  2023038329
- Rafaela Kesia Ferreira da Silva - 2025019879
- Raissa Gonçalves Diniz - 2022055823

# 3. Descrição dos dados (qual a URL? qual o domínio? como os dados foram processados?)

- Link: https://dados.gov.br/dados/conjuntos-dados/alunos-de-iniciacao-cientifica.
- Descrição: O dataset escolhido reúne informações sobre os estudantes envolvidos em projetos de iniciação científica na Universidade Federal do Triângulo Mineiro, incluindo dados sobre alunos, curso, vigência, projeto, orientador e outros. Dentre os arquivos csv, escolhemos o de nome “Informações Acadêmicas - Alunos De Iniciação Científica - 2025”, catalogado em 28/03/2025.
- Processamento dos Dados:

In [1]:
import csv
import pandas as pd
import sqlite3
import re
from unidecode import unidecode

In [2]:
def fetch(query, conn, formatted=True):
    """Executa query e retorna DataFrame ou lista de tuplas"""
    cur = conn.cursor()
    cur.execute(query)
    rs = cur.fetchall()
    columns = [desc[0] for desc in cur.description]
    return pd.DataFrame(rs, columns=columns) if formatted else rs

def show_tables(conn):
    """Lista todas as tabelas do banco"""
    return [x[0] for x in fetch('SELECT tbl_name FROM sqlite_master WHERE type="table"', conn, False)]

def shape(table, conn):
    """Retorna dimensões da tabela (linhas, colunas)"""
    nrows = fetch(f'SELECT COUNT(*) FROM {table}', conn, False)[0][0]
    ncols = fetch(f'SELECT COUNT(*) FROM pragma_table_info("{table}")', conn, False)[0][0]
    return (nrows, ncols)

def desc(table, conn):
    """Lista nomes das colunas"""
    cur = conn.cursor()
    cur.execute(f'PRAGMA table_info("{table}")')
    columns = [row[1] for row in cur.fetchall()]
    return columns

def info(table, conn):
    """Informações detalhadas sobre a tabela (análogo ao pandas.info())"""
    df1 = fetch(f'PRAGMA table_info("{table}")', conn)
    columns = desc(table, conn)
    
    # total de entradas por coluna
    counts = ', '.join([f'COUNT(*) AS "{column}"' for column in columns])
    df2 = fetch(f'SELECT {counts} FROM "{table}"', conn).transpose()
    df2.columns = ['count']
    
    # entradas não-nulas por coluna
    counts = ', '.join([f'COUNT("{column}") AS "{column}"' for column in columns])
    df3 = fetch(f'SELECT {counts} FROM "{table}"', conn).transpose()
    df3.columns = ['notnull count']

    # entradas únicas não-nulas por coluna
    counts = ', '.join([f'COUNT(DISTINCT "{column}") AS "{column}"' for column in columns])
    df4 = fetch(f'SELECT {counts} FROM "{table}"', conn).transpose()
    df4.columns = ['unique count']
    
    return df1.merge(df2, left_on='name', right_index=True) \
            .merge(df3, left_on='name', right_index=True) \
            .merge(df4, left_on='name', right_index=True)

In [3]:
# Dataset fonte
source = 'ALUNADO IC 2024.csv'

# Conectar ao banco SQLite
conn = sqlite3.connect('ic-uftm.db')
cur = conn.cursor()

# Configurar PRAGMA para ingestão mais rápida
cur.execute('PRAGMA synchronous = OFF')
cur.execute('PRAGMA journal_mode = MEMORY')

# Tamanho do batch para processamento
BATCH_SIZE = 1000

# Iniciar transação
cur.execute('BEGIN TRANSACTION')

# Ler CSV e inserir em batches
with open(source, 'r', encoding='latin-1') as f:
    reader = csv.reader(f, delimiter=';')
    header = next(reader)
    
    # Criar tabela com nomes apropriados
    columns = [f'"{column}"' for column in header]
    create = f'CREATE TABLE IF NOT EXISTS Source ({", ".join(columns)})'
    cur.execute(create)
    
    # Preparar INSERT
    placeholders = ', '.join(['?'] * len(header))
    insert = f'INSERT INTO Source VALUES ({placeholders})'
    
    # Processar em batches
    batch = []
    for row in reader:
        batch.append(row)
        if len(batch) == BATCH_SIZE:
            cur.executemany(insert, batch)
            batch = []
    
    # Inserir linhas restantes
    if batch:
        cur.executemany(insert, batch)

# Converter valores vazios em NULL
update = f'UPDATE Source SET ' + ','.join([f'{column} = NULLIF({column}, "")' for column in columns])
cur.execute(update)

# Commit
conn.commit()

# Restaurar PRAGMA
cur.execute('PRAGMA synchronous = FULL')
cur.execute('PRAGMA journal_mode = DELETE')

print("Dados carregados com sucesso!")

Dados carregados com sucesso!


In [4]:
print("Tabelas no banco:")
show_tables(conn)

Tabelas no banco:


['Source']

In [5]:
print(f"Shape da tabela Source: {shape('Source', conn)}")

Shape da tabela Source: (566, 9)


In [6]:
info('Source', conn)

,cid,name,type,notnull,dflt_value,pk,count,notnull count,unique count
0,0,Nome do Aluno,,0,None,0,566,566,537
1,1,Código do Projeto (nº Matrícula),,0,None,0,566,566,565
2,2,Curso do Aluno,,0,None,0,566,566,31
3,3,Data de Início do Projeto,,0,None,0,566,566,101
4,4,Data de Fim do Projeto,,0,None,0,566,566,103
5,5,Fomento do Aluno,,0,None,0,566,566,8
6,6,Título do Projeto,,0,None,0,566,566,565
7,7,Orientador,,0,None,0,566,566,269
8,8,Edital,,0,None,0,566,566,6


In [7]:
print("Primeiras 5 linhas:")
fetch('SELECT * FROM Source LIMIT 5', conn)

Primeiras 5 linhas:


,Nome do Aluno,Código do Projeto (nº Matrícula),Curso do Aluno,Data de Início do Projeto,Data de Fim do Projeto,Fomento do Aluno,Título do Projeto,Orientador,Edital
0,Adelson Antonio Januario Junior,5082,EDUCAÇÃO FÍSICA (Bacharelado),01/10/2023,30/09/2024,PIBIC/FAPEMIG,EFEITO DO TREINAMENTO RESISTIDO NA COMPOSIÇÃO ...,CLAUDIO DE OLIVEIRA ASSUMPCAO,Edital 22/2023 - Iniciação Científica e Tecnol...
1,Adônis Rossetti,4178,MEDICINA (Bacharelado),21/03/2023,20/03/2024,Sem bolsa,O impacto das mídias sociais na visão corporal...,VITOR HUGO DE OLIVEIRA,Edital 01/2023 - Iniciação Científica sem bols...
2,Agatha Ferreira Souza,8155,BIOMEDICINA (Bacharelado),01/10/2024,30/09/2025,PIBIC/FAPEMIG,Análise pangenômica e de plasticidade do gêner...,Wellington Francisco Rodrigues,Edital 20/2024 - Iniciação Científica e Tecnol...
3,Agnaldo José Braga Junior,4831,ENGENHARIA AMBIENTAL (Bacharelado),01/09/2023,31/08/2024,PIBIC-Af/CNPq,Efeitos da sujidade no desempenho de placas fo...,Julio Cesar De Souza Inacio Goncalves,Edital 14/2023 - Iniciação Científica e Tecnol...
4,Alan de Castro Barbosa,5137,MEDICINA (Bacharelado),01/10/2023,30/09/2024,PIBIC/FAPEMIG,Perfil de citocinas associadas a polimorfismos...,Rodrigo Anselmo Cazzaniga,Edital 22/2023 - Iniciação Científica e Tecnol...


In [8]:
print("\nCONTAGEM DE VALORES ÚNICOS\n")
print(f"Alunos únicos: {fetch('SELECT COUNT(DISTINCT [Nome do Aluno]) FROM Source', conn, False)[0][0]}")
print(f"Projetos únicos: {fetch('SELECT COUNT(DISTINCT [Código do Projeto (nº Matrícula)]) FROM Source', conn, False)[0][0]}")
print(f"Orientadores únicos: {fetch('SELECT COUNT(DISTINCT [Orientador]) FROM Source', conn, False)[0][0]}")
print(f"Cursos únicos: {fetch('SELECT COUNT(DISTINCT [Curso do Aluno]) FROM Source', conn, False)[0][0]}")
print(f"Tipos de Fomento únicos: {fetch('SELECT COUNT(DISTINCT [Fomento do Aluno]) FROM Source', conn, False)[0][0]}")
print(f"Total de participações (linhas): {fetch('SELECT COUNT(*) FROM Source', conn, False)[0][0]}")


CONTAGEM DE VALORES ÚNICOS

Alunos únicos: 537
Projetos únicos: 565
Orientadores únicos: 269
Cursos únicos: 31
Tipos de Fomento únicos: 8
Total de participações (linhas): 566


In [9]:
# Ver todos os cursos únicos
print("Cursos únicos:")
fetch('SELECT DISTINCT "Curso do Aluno" FROM Source ORDER BY "Curso do Aluno"', conn)

Cursos únicos:


,Curso do Aluno
0,9º ano do Ensino Fundamental
1,AGRONOMIA (Bacharelado)
2,BIOMEDICINA (Bacharelado)
3,CIÊNCIAS BIOLÓGICAS (Licenciatura)
4,EDUCAÇÃO FÍSICA (Bacharelado)
5,ENFERMAGEM (Bacharelado)
6,ENGENHARIA AMBIENTAL (Bacharelado)
7,ENGENHARIA CIVIL (Bacharelado)
8,ENGENHARIA DE ALIMENTOS (Bacharelado)
9,ENGENHARIA DE PRODUÇÃO (Bacharelado)


In [10]:
# Ver todos os tipos de fomento
print("Tipos de Fomento:")
fetch('SELECT "Fomento do Aluno", COUNT(*) as Qtd FROM Source GROUP BY "Fomento do Aluno" ORDER BY Qtd DESC', conn)

Tipos de Fomento:


,Fomento do Aluno,Qtd
0,PIBIC/FAPEMIG,218
1,Sem bolsa,208
2,PIBIC/CNPq,79
3,PIBIC-Af/CNPq,29
4,PIBIC/UFTM,15
5,PIBITI/CNPq,7
6,PIBIC-AF/UFTM,7
7,PIBITI/UFTM,3


In [11]:
# Funções auxiliares para fazer as Transformações

def gerar_email(nome):
    """Gera email a partir do nome: primeironome.segundonome@gmail.com"""
    if not nome or pd.isna(nome):
        return None
    
    # Remover acentos e converter para minúsculas
    nome_limpo = unidecode(nome.lower())
    
    # Dividir em palavras
    partes = nome_limpo.split()
    
    if len(partes) >= 2:
        return f"{partes[0]}.{partes[1]}@gmail.com"
    elif len(partes) == 1:
        return f"{partes[0]}@gmail.com"
    else:
        return "email@gmail.com"
    
def mapear_departamento(curso):
    """Mapeia curso para departamento do orientador"""
    mapeamento = {
        # Departamento de Ciências Biológicas e da Saúde
        'BIOMEDICINA (Bacharelado)': 'Departamento de Ciências Biológicas e da Saúde',
        'CIÊNCIAS BIOLÓGICAS (Licenciatura)': 'Departamento de Ciências Biológicas e da Saúde',
        'EDUCAÇÃO FÍSICA (Bacharelado)': 'Departamento de Ciências Biológicas e da Saúde',
        'ENFERMAGEM (Bacharelado)': 'Departamento de Ciências Biológicas e da Saúde',
        'FISIOTERAPIA (Bacharelado)': 'Departamento de Ciências Biológicas e da Saúde',
        'MEDICINA (Bacharelado)': 'Departamento de Ciências Biológicas e da Saúde',
        'NUTRIÇÃO (Bacharelado)': 'Departamento de Ciências Biológicas e da Saúde',
        'TERAPIA OCUPACIONAL (Bacharelado)': 'Departamento de Ciências Biológicas e da Saúde',
        'TÉCNICO EM RADIOLOGIA': 'Departamento de Ciências Biológicas e da Saúde',
        'TÉCNICO EM FARMÁCIA': 'Departamento de Ciências Biológicas e da Saúde',
        
        # Departamento de Engenharia e Tecnologia
        'AGRONOMIA (Bacharelado)': 'Departamento de Engenharia e Tecnologia',
        'ENGENHARIA AMBIENTAL (Bacharelado)': 'Departamento de Engenharia e Tecnologia',
        'ENGENHARIA CIVIL (Bacharelado)': 'Departamento de Engenharia e Tecnologia',
        'ENGENHARIA DE ALIMENTOS (Bacharelado)': 'Departamento de Engenharia e Tecnologia',
        'ENGENHARIA DE PRODUÇÃO (Bacharelado)': 'Departamento de Engenharia e Tecnologia',
        'ENGENHARIA ELÉTRICA (Bacharelado)': 'Departamento de Engenharia e Tecnologia',
        'ENGENHARIA MECÂNICA (Bacharelado)': 'Departamento de Engenharia e Tecnologia',
        'ENGENHARIA QUÍMICA (Bacharelado)': 'Departamento de Engenharia e Tecnologia',
        
        # Departamento de Ciências Exatas e Naturais
        'FÍSICA (Licenciatura)': 'Departamento de Ciências Exatas e Naturais',
        'QUÍMICA (Licenciatura)': 'Departamento de Ciências Exatas e Naturais',
        'MATEMÁTICA (Licenciatura)': 'Departamento de Ciências Exatas e Naturais',
        'LICENCIATURA EM EDUCAÇÃO DO CAMPO – Ciências da Natureza': 'Departamento de Ciências Exatas e Naturais',
        'LICENCIATURA EM EDUCAÇÃO DO CAMPO – Matemática': 'Departamento de Ciências Exatas e Naturais',
        
        # Departamento de Humanidades e Educação
        'LETRAS – PORTUGUÊS E ESPANHOL (Licenciatura)': 'Departamento de Humanidades e Educação',
        'LETRAS – PORTUGUÊS E INGLÊS (Licenciatura)': 'Departamento de Humanidades e Educação',
        'HISTÓRIA (Licenciatura)': 'Departamento de Humanidades e Educação',
        'GEOGRAFIA (Licenciatura)': 'Departamento de Humanidades e Educação',
        'SOCIOLOGIA (Bacharelado)': 'Departamento de Humanidades e Educação',
        'SERVIÇO SOCIAL (Bacharelado)': 'Departamento de Humanidades e Educação',
        
        # Departamento de Educação Básica
        'Ensino Fundamental': 'Departamento de Educação Básica',
        '9º ano do Ensino Fundamental': 'Departamento de Educação Básica',
        'Ensino Médio': 'Departamento de Educação Básica'
    }
    
    return mapeamento.get(curso, 'Departamento não identificado')

def mapear_centro_academico(curso):
    """Mapeia curso para centro acadêmico"""
    mapeamento = {
        # FAFICH
        'HISTÓRIA (Licenciatura)': 'FAFICH',
        'GEOGRAFIA (Licenciatura)': 'FAFICH',
        'SOCIOLOGIA (Bacharelado)': 'FAFICH',
        'SERVIÇO SOCIAL (Bacharelado)': 'FAFICH',
        'LETRAS – PORTUGUÊS E ESPANHOL (Licenciatura)': 'FAFICH',
        'LETRAS – PORTUGUÊS E INGLÊS (Licenciatura)': 'FAFICH',
        'LICENCIATURA EM EDUCAÇÃO DO CAMPO – Ciências da Natureza': 'FAFICH',
        'LICENCIATURA EM EDUCAÇÃO DO CAMPO – Matemática': 'FAFICH',
        
        # Escola de Engenharia
        'ENGENHARIA AMBIENTAL (Bacharelado)': 'Escola de Engenharia',
        'ENGENHARIA CIVIL (Bacharelado)': 'Escola de Engenharia',
        'ENGENHARIA DE ALIMENTOS (Bacharelado)': 'Escola de Engenharia',
        'ENGENHARIA DE PRODUÇÃO (Bacharelado)': 'Escola de Engenharia',
        'ENGENHARIA ELÉTRICA (Bacharelado)': 'Escola de Engenharia',
        'ENGENHARIA MECÂNICA (Bacharelado)': 'Escola de Engenharia',
        'ENGENHARIA QUÍMICA (Bacharelado)': 'Escola de Engenharia',
        
        # ICEx
        'MATEMÁTICA (Licenciatura)': 'ICEx',
        'FÍSICA (Licenciatura)': 'ICEx',
        'QUÍMICA (Licenciatura)': 'ICEx',
        
        # ICB
        'CIÊNCIAS BIOLÓGICAS (Licenciatura)': 'ICB',
        'BIOMEDICINA (Bacharelado)': 'ICB',
        
        # Faculdade de Medicina
        'MEDICINA (Bacharelado)': 'Faculdade de Medicina',
        'FISIOTERAPIA (Bacharelado)': 'Faculdade de Medicina',
        'ENFERMAGEM (Bacharelado)': 'Faculdade de Medicina',
        'NUTRIÇÃO (Bacharelado)': 'Faculdade de Medicina',
        'TERAPIA OCUPACIONAL (Bacharelado)': 'Faculdade de Medicina',
        'EDUCAÇÃO FÍSICA (Bacharelado)': 'Faculdade de Medicina',
        'TÉCNICO EM RADIOLOGIA': 'Faculdade de Medicina',
        'TÉCNICO EM FARMÁCIA': 'Faculdade de Medicina',
        
        # Instituto de Ciências Agrárias
        'AGRONOMIA (Bacharelado)': 'Instituto de Ciências Agrárias',
        
        # Centro de Ensino Básico
        'Ensino Fundamental': 'Centro de Ensino Básico e Formação de Professores',
        '9º ano do Ensino Fundamental': 'Centro de Ensino Básico e Formação de Professores',
        'Ensino Médio': 'Centro de Ensino Básico e Formação de Professores'
    }
    
    return mapeamento.get(curso, 'Centro não identificado')

def classificar_fomento(fomento):
    """Classifica o tipo de fomento (Federal, Estadual, Institucional)"""
    if not fomento or pd.isna(fomento) or fomento == 'Sem bolsa':
        return None
    
    if 'CNPq' in fomento:
        return 'Federal'
    elif 'FAPEMIG' in fomento:
        return 'Estadual'
    elif 'UFTM' in fomento:
        return 'Institucional'
    else:
        return 'Outro'

In [ ]:
# Criar tabelas normalizadas
ddl = [
    '''
    CREATE TABLE IF NOT EXISTS Curso (
        ID_Curso INTEGER PRIMARY KEY AUTOINCREMENT,
        Nome_Curso TEXT NOT NULL UNIQUE,
        Centro_Academico TEXT
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS Aluno (
        ID_Aluno INTEGER PRIMARY KEY AUTOINCREMENT,
        Nome TEXT NOT NULL,
        Email TEXT,
        ID_Curso INTEGER NOT NULL,
        FOREIGN KEY (ID_Curso) REFERENCES Curso(ID_Curso)
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS Orientador (
        ID_Orientador INTEGER PRIMARY KEY AUTOINCREMENT,
        Nome TEXT NOT NULL UNIQUE,
        Departamento TEXT
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS Projeto (
        ID_Projeto INTEGER PRIMARY KEY AUTOINCREMENT,
        Nome_Projeto TEXT NOT NULL,
        Nome_Edital TEXT,
        ID_Orientador INTEGER NOT NULL,
        Codigo_Referencia INTEGER,
        FOREIGN KEY (ID_Orientador) REFERENCES Orientador(ID_Orientador)
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS Orgao_Fomento (
        ID_Fomento INTEGER PRIMARY KEY AUTOINCREMENT,
        Nome_Fomento TEXT NOT NULL UNIQUE,
        Tipo TEXT
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS Participa (
        ID_Aluno INTEGER NOT NULL,
        ID_Projeto INTEGER NOT NULL,
        Data_Inicio_Vigencia DATE NOT NULL,
        Data_Termino_Vigencia DATE NOT NULL,
        ID_Fomento INTEGER,
        PRIMARY KEY (ID_Aluno, ID_Projeto),
        FOREIGN KEY (ID_Aluno) REFERENCES Aluno(ID_Aluno),
        FOREIGN KEY (ID_Projeto) REFERENCES Projeto(ID_Projeto),
        FOREIGN KEY (ID_Fomento) REFERENCES Orgao_Fomento(ID_Fomento)
    );
    '''
]

for q in ddl:
    cur.execute(q)

conn.commit()
print("Esquema normalizado criado!")

Esquema normalizado criado!


In [13]:
# Verificar tabelas criadas
print("Tabelas no banco:")
print(show_tables(conn))

Tabelas no banco:
['Source', 'Curso', 'sqlite_sequence', 'Aluno', 'Orientador', 'Projeto', 'Orgao_Fomento', 'Participa']


In [14]:
# Migrar Curso

cur.execute('DELETE FROM Curso')

# Buscar cursos únicos
cursos = fetch('SELECT DISTINCT "Curso do Aluno" FROM Source WHERE "Curso do Aluno" IS NOT NULL', conn, False)

# Inserir com centro acadêmico
for (curso,) in cursos:
    centro = mapear_centro_academico(curso)
    cur.execute('INSERT INTO Curso (Nome_Curso, Centro_Academico) VALUES (?, ?)', (curso, centro))

conn.commit()
print(f"{len(cursos)} cursos migrados!")
info('Curso', conn)

31 cursos migrados!


,cid,name,type,notnull,dflt_value,pk,count,notnull count,unique count
0,0,ID_Curso,INTEGER,0,None,1,31,31,31
1,1,Nome_Curso,TEXT,1,None,0,31,31,31
2,2,Centro_Academico,TEXT,0,None,0,31,31,8


In [15]:
# Migrar Orientador

cur.execute('DELETE FROM Orientador')

# Buscar orientadores únicos com seus cursos
orientadores_cursos = fetch('''
    SELECT DISTINCT Orientador, "Curso do Aluno" 
    FROM Source 
    WHERE Orientador IS NOT NULL
''', conn, False)

# Agrupar por orientador e pegar o primeiro curso (para determinar departamento)
orientadores_dict = {}
for orientador, curso in orientadores_cursos:
    if orientador not in orientadores_dict:
        orientadores_dict[orientador] = curso

# Inserir com departamento
for orientador, curso in orientadores_dict.items():
    departamento = mapear_departamento(curso)
    cur.execute('INSERT INTO Orientador (Nome, Departamento) VALUES (?, ?)', (orientador, departamento))

conn.commit()
print(f"{len(orientadores_dict)} orientadores migrados!")
info('Orientador', conn)

269 orientadores migrados!


,cid,name,type,notnull,dflt_value,pk,count,notnull count,unique count
0,0,ID_Orientador,INTEGER,0,None,1,269,269,269
1,1,Nome,TEXT,1,None,0,269,269,269
2,2,Departamento,TEXT,0,None,0,269,269,6


In [16]:
# Migrar Órgão de Fomento

cur.execute('DELETE FROM Orgao_Fomento')

# Buscar tipos de fomento únicos (excluindo "Sem bolsa")
fomentos = fetch('''
    SELECT DISTINCT "Fomento do Aluno" 
    FROM Source 
    WHERE "Fomento do Aluno" IS NOT NULL 
    AND "Fomento do Aluno" != "Sem bolsa"
''', conn, False)

# Inserir com tipo
for (fomento,) in fomentos:
    tipo = classificar_fomento(fomento)
    cur.execute('INSERT INTO Orgao_Fomento (Nome_Fomento, Tipo) VALUES (?, ?)', (fomento, tipo))

conn.commit()
print(f"{len(fomentos)} órgãos de fomento migrados!")
info('Orgao_Fomento', conn)

7 órgãos de fomento migrados!


,cid,name,type,notnull,dflt_value,pk,count,notnull count,unique count
0,0,ID_Fomento,INTEGER,0,None,1,7,7,7
1,1,Nome_Fomento,TEXT,1,None,0,7,7,7
2,2,Tipo,TEXT,0,None,0,7,7,3


In [17]:
# Migrar Aluno (com FK para Curso)

cur.execute('DELETE FROM Aluno')

# Buscar alunos únicos com seus cursos
alunos_cursos = fetch('''
    SELECT DISTINCT "Nome do Aluno", "Curso do Aluno"
    FROM Source 
    WHERE "Nome do Aluno" IS NOT NULL
''', conn, False)

# Inserir com email gerado e curso
for nome, curso in alunos_cursos:
    email = gerar_email(nome)
    # Buscar ID do curso
    id_curso = fetch(f'SELECT ID_Curso FROM Curso WHERE Nome_Curso = "{curso}"', conn, False)[0][0]
    cur.execute('INSERT INTO Aluno (Nome, Email, ID_Curso) VALUES (?, ?, ?)', (nome, email, id_curso))

conn.commit()
print(f"{len(alunos_cursos)} alunos migrados!")
info('Aluno', conn)

537 alunos migrados!


,cid,name,type,notnull,dflt_value,pk,count,notnull count,unique count
0,0,ID_Aluno,INTEGER,0,None,1,537,537,537
1,1,Nome,TEXT,1,None,0,537,537,537
2,2,Email,TEXT,0,None,0,537,537,488
3,3,ID_Curso,INTEGER,1,None,0,537,537,31


In [ ]:
# Migrar Projeto (com FK para Orientador)

cur.execute('DELETE FROM Projeto')

# Migrar projetos com orientadores
cur.execute('''
INSERT INTO Projeto (Nome_Projeto, Nome_Edital, ID_Orientador, Codigo_Referencia)
SELECT DISTINCT 
    s."Título do Projeto",
    s."Edital",
    o.ID_Orientador,
    CAST(s."Código do Projeto (nº Matrícula)" AS INTEGER)
FROM Source s
JOIN Orientador o ON s.Orientador = o.Nome
WHERE s."Título do Projeto" IS NOT NULL
''')

conn.commit()
print("Projetos migrados!")
info('Projeto', conn)

Projetos migrados!


,cid,name,type,notnull,dflt_value,pk,count,notnull count,unique count
0,0,ID_Projeto,INTEGER,0,None,1,566,566,566
1,1,Nome_Projeto,TEXT,1,None,0,566,566,565
2,2,Numero_Edital,INTEGER,0,None,0,566,566,1
3,3,ID_Orientador,INTEGER,1,None,0,566,566,269
4,4,Codigo_Referencia,INTEGER,0,None,0,566,566,565


In [ ]:
# Migrar Participa (Aluno M:N Projeto)

cur.execute('DELETE FROM Participa')

# Migrar participações
cur.execute('''
INSERT INTO Participa (
    ID_Aluno, 
    ID_Projeto, 
    Data_Inicio_Vigencia, 
    Data_Termino_Vigencia, 
    ID_Fomento
)
SELECT 
    a.ID_Aluno,
    p.ID_Projeto, -- O NOVO ID_Projeto (da tabela Projeto)
    MIN(s."Data de Início do Projeto"),
    MAX(s."Data de Fim do Projeto"),
    MAX(of.ID_Fomento)
FROM Source s
JOIN Aluno a ON s."Nome do Aluno" = a.Nome
JOIN Orientador o ON s.Orientador = o.Nome
LEFT JOIN Orgao_Fomento of ON s."Fomento do Aluno" = of.Nome_Fomento

JOIN Projeto p ON s."Título do Projeto" = p.Nome_Projeto AND p.ID_Orientador = o.ID_Orientador

WHERE s."Código do Projeto (nº Matrícula)" IS NOT NULL
  AND s."Nome do Aluno" IS NOT NULL
  
GROUP BY
    a.ID_Aluno,
    p.ID_Projeto
''')

conn.commit()
print("Participações (Aluno M:N Projeto) migradas!")
info('Participa', conn)

Participações (Aluno M:N Projeto) migradas!


,cid,name,type,notnull,dflt_value,pk,count,notnull count,unique count
0,0,ID_Aluno,INTEGER,1,None,1,566,566,537
1,1,ID_Projeto,INTEGER,1,None,2,566,566,566
2,2,Data_Inicio_Vigencia,DATE,1,None,0,566,566,100
3,3,Data_Termino_Vigencia,DATE,1,None,0,566,566,102
4,4,ID_Fomento,INTEGER,0,None,0,566,359,7


# 4. Diagrama ER

# 5. Diagrama relacional

# 6. Consultas

## 6.1 Duas consultas envolvendo seleção e projeção

### 6.1.1 Consulta 1

### 6.1.2 Consulta 2

## 6.2 Três consultas envolvendo junção de duas relações

### 6.2.1 Consulta 3

### 6.2.2 Consulta 4

### 6.2.3 Consulta 5

## 6.3 Três consultas envolvendo junção de três ou mais relações

### 6.3.1 Consulta 6

### 6.3.2 Consulta 7

### 6.3.3 Consulta 8

## 6.4 Duas consultas envolvendo agregação sobre junção de duas ou mais relações

### 6.4.1 Consulta 9

### 6.4.2 Consulta 10

# 7. Autoavaliação dos membros